In [1]:
import pandas as pd
import numpy as np
import os
currency = 7.13
post_discount_rate = 0.77
during_discount_rate = 0.68
df_with_total = pd.DataFrame()
# --- Load raw data ---
df_raw_data_current_week = pd.read_excel('/Users/jackzhao/Desktop/auto_tracker_testing/LM_raw_data_current_week.xlsx')


path_previous = '/Users/jackzhao/Desktop/auto_tracker_testing/LM_raw_data_previous_week.xlsx'

def internal_data_tracker(currency, df_raw_data, post_discount_rate, during_discount_rate, df_raw_data_path_previous_week=None):

    df_raw_data['TA'] = df_raw_data['TA'].astype(str)
    df_raw_data['Phasing'] = df_raw_data['Phasing'].astype(str)
    df_raw_data['花费'] = pd.to_numeric(df_raw_data['花费'], errors='coerce').fillna(0)
    df_raw_data['曝光次数'] = pd.to_numeric(df_raw_data['曝光次数'], errors='coerce').fillna(0)
    df_raw_data['点击次数'] = pd.to_numeric(df_raw_data['点击次数'], errors='coerce').fillna(0)
    df_with_total = pd.DataFrame([
    {"Campaign Name": "LM FY25 Growth Hunting - Q4", "Brand": "LM", "Date": "4/1–6/30", "Phasing": "Post", "TA GROUP": "TA1", "TA 定义": "TA1 定义", "Media Budget": "$666,077", "Top-up Internet": "¥4,277,930"},
    {"Campaign Name": "LM FY25 Growth Hunting - Q4", "Brand": "LM", "Date": "4/1–6/30", "Phasing": "Post", "TA GROUP": "TA2", "TA 定义": "TA2 定义", "Media Budget": "", "Top-up Internet": ""},
    {"Campaign Name": "LM FY25 Growth Hunting - Q4", "Brand": "LM", "Date": "4/1–6/30", "Phasing": "Post", "TA GROUP": "TA3", "TA 定义": "TA3 定义", "Media Budget": "", "Top-up Internet": ""},
    {"Campaign Name": "LM FY25 Growth Hunting - Q4", "Brand": "LM", "Date": "4/1–6/30", "Phasing": "Post", "TA GROUP": "TA4", "TA 定义": "TA4 定义", "Media Budget": "", "Top-up Internet": ""},
    {"Campaign Name": "LM FY25 Growth Hunting - Q4", "Brand": "LM", "Date": "4/1–6/30", "Phasing": "Post", "TA GROUP": "TA5", "TA 定义": "TA5 定义", "Media Budget": "", "Top-up Internet": ""},
    {"Campaign Name": "LM FY25 Growth Hunting - Q4", "Brand": "LM", "Date": "4/1–6/30", "Phasing": "Post", "TA GROUP": "Super TA", "TA 定义": "Super TA 定义", "Media Budget": "", "Top-up Internet": ""},
    {"Campaign Name": "LM FY25 Growth Hunting - Q4", "Brand": "LM", "Date": "4/1–6/30", "Phasing": "Post", "TA GROUP": "Post Total", "TA 定义": "", "Media Budget": "$666,077", "Top-up Internet": "¥4,277,930"},
    {"Campaign Name": "LM FY25 Growth Hunting - Q4", "Brand": "LM", "Date": "4/1–6/30", "Phasing": "During", "TA GROUP": "During", "TA 定义": "Hainan Traveler", "Media Budget": "$311,560", "Top-up Internet": "¥2,000,800"}])
    spend_values = []
    impression_values = []
    click_values = []
    

    for i, row in df_with_total.iterrows():
        ta_group = row['TA GROUP']
        phasing = row['Phasing']

        if ta_group in ["Post Total", "Total"]:
            spend_values.append(0)
            impression_values.append(0)
            click_values.append(0)
            continue
        if ta_group == "During" and phasing == "During":
            total_spend = df_raw_data.loc[df_raw_data['Phasing'] == "During", '花费'].sum()
            total_impr = df_raw_data.loc[df_raw_data['Phasing'] == "During", '曝光次数'].sum()
            total_click = df_raw_data.loc[df_raw_data['Phasing'] == "During", '点击次数'].sum()
            spend_values.append(total_spend)
            impression_values.append(total_impr)
            click_values.append(total_click)
            continue
        total_spend = df_raw_data.loc[(df_raw_data['TA'] == ta_group) & (df_raw_data['Phasing'] == phasing), '花费'].sum()

        total_impr = df_raw_data.loc[(df_raw_data['TA'] == ta_group) & (df_raw_data['Phasing'] == phasing), '曝光次数'].sum()

        total_click = df_raw_data.loc[(df_raw_data['TA'] == ta_group) & (df_raw_data['Phasing'] == phasing), '点击次数'].sum()
        spend_values.append(total_spend)
        impression_values.append(total_impr)
        click_values.append(total_click)
    df_with_total['Spend￥_num'] = spend_values
    df_with_total['impression_num'] = impression_values
    df_with_total['click_num'] = click_values

    # --- Post Total values ---
    post_total_spend = df_with_total.loc[df_with_total['Phasing'] == 'Post', 'Spend￥_num'].sum()
    post_total_impr = df_with_total.loc[df_with_total['Phasing'] == 'Post', 'impression_num'].sum()
    post_total_click = df_with_total.loc[df_with_total['Phasing'] == 'Post', 'click_num'].sum()

    df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'Spend￥_num'] = post_total_spend
    df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'impression_num'] = post_total_impr
    df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'click_num'] = post_total_click

    # --- Total row values ---
    during_row = df_with_total.loc[df_with_total['TA GROUP'] == 'During']

    total_spend = post_total_spend + during_row['Spend￥_num'].values[0]
    total_impr = post_total_impr + during_row['impression_num'].values[0]
    total_click = post_total_click + during_row['click_num'].values[0]
    def parse_currency(x):
        try:
            return float(x.replace("$", "").replace(",", ""))
        except:
            return 0

    def parse_yen(x):
        try:
            return float(x.replace("¥", "").replace(",", ""))
        except:
            return 0

    post_media_budget = parse_currency(df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'Media Budget'].values[0])
    during_media_budget = parse_currency(during_row['Media Budget'].values[0])
    total_media_budget = post_media_budget + during_media_budget

    post_topup = parse_yen(df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'Top-up Internet'].values[0])
    during_topup = parse_yen(during_row['Top-up Internet'].values[0])
    total_topup = post_topup + during_topup

    # Format
    total_media_budget_str = f"${total_media_budget:,.0f}"
    total_topup_str = f"¥{total_topup:,.0f}"
    total_row = {
        "Campaign Name": "",
        "Brand": "",
        "Date": "",
        "Phasing": "",
        "TA GROUP": "Total",
        "TA 定义": "",
        "Media Budget": total_media_budget_str,
        "Top-up Internet": total_topup_str,
        "Spend￥_num": total_spend,
        "impression_num": total_impr,
        "click_num": total_click
    }
    df_with_total = pd.concat([df_with_total, pd.DataFrame([total_row])], ignore_index=True)


    #  --- Keep Spend$ as raw float ---
    df_with_total['Spend$'] = df_with_total['Spend￥_num'] / currency

    # --- Keep Spend￥ as raw float ---
    df_with_total['Spend￥'] = df_with_total['Spend￥_num']

    # --- Keep impression and click as integer ---
    df_with_total['impression'] = df_with_total['impression_num']
    df_with_total['click'] = df_with_total['click_num']

    def calc_ctr(row):
        try:
            if row['impression_num'] > 0:
                return row['click_num'] / row['impression_num']
            else:
                return 0
        except:
            return 0
    df_with_total['CTR'] = df_with_total.apply(calc_ctr, axis=1)

    # --- Drop numeric helpers only if you want (now, we can keep them!) ---
    # df_with_total = df_with_total.drop(columns=['Spend￥_num', 'impression_num', 'click_num', 'CTR_num'], errors='ignore')

    # --- Drop numeric helpers ---
    df_with_total = df_with_total.drop(columns=['Spend￥_num', 'impression_num', 'click_num'], errors='ignore')

    ### bock 2
    
    pv_values = []
    uv_values = []

    for i, row in df_with_total.iterrows():
        ta_group = row['TA GROUP']
        phasing = row['Phasing']

        if ta_group in ["Post Total", "Total"]:
            pv_values.append(0)
            uv_values.append(0)
            continue

        if ta_group == "During" and phasing == "During":
            total_pv = df_raw_data.loc[df_raw_data['Phasing'] == "During", df_raw_data.columns[13]].sum()  # Column N
            total_uv = df_raw_data.loc[df_raw_data['Phasing'] == "During", df_raw_data.columns[14]].sum()  # Column O
            pv_values.append(total_pv)
            uv_values.append(total_uv)
            continue

        total_pv = df_raw_data.loc[
            (df_raw_data['TA'] == ta_group) & (df_raw_data['Phasing'] == phasing), df_raw_data.columns[13]
        ].sum()

        total_uv = df_raw_data.loc[
            (df_raw_data['TA'] == ta_group) & (df_raw_data['Phasing'] == phasing), df_raw_data.columns[14]
        ].sum()

        pv_values.append(total_pv)
        uv_values.append(total_uv)

    df_with_total['PV'] = pv_values
    df_with_total['UV'] = uv_values

    # --- Post Total row values ---
    post_total_pv = df_with_total.loc[df_with_total['Phasing'] == 'Post', 'PV'].sum()
    post_total_uv = df_with_total.loc[df_with_total['Phasing'] == 'Post', 'UV'].sum()
    df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'PV'] = post_total_pv
    df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'UV'] = post_total_uv

    # --- Total row values ---
    during_row = df_with_total.loc[df_with_total['TA GROUP'] == 'During']

    total_pv = post_total_pv + during_row['PV'].values[0]
    total_uv = post_total_uv + during_row['UV'].values[0]

    df_with_total.loc[df_with_total['TA GROUP'] == 'Total', 'PV'] = total_pv
    df_with_total.loc[df_with_total['TA GROUP'] == 'Total', 'UV'] = total_uv
    
    cart_values = []
    order_values = []

    for i, row in df_with_total.iterrows():
        ta_group = row['TA GROUP']
        phasing = row['Phasing']

        if ta_group in ["Post Total", "Total"]:
            cart_values.append(0)
            order_values.append(0)
            continue

        if ta_group == "During" and phasing == "During":
            total_cart = df_raw_data.loc[df_raw_data['Phasing'] == "During", '加入购物车次数'].sum()
            total_order = df_raw_data.loc[df_raw_data['Phasing'] == "During", df_raw_data.columns[11]].sum()  # Corrected!
            cart_values.append(total_cart)
            order_values.append(total_order)
            continue

        total_cart = df_raw_data.loc[
            (df_raw_data['TA'] == ta_group) & (df_raw_data['Phasing'] == phasing), '加入购物车次数'
        ].sum()

        total_order = df_raw_data.loc[
            (df_raw_data['TA'] == ta_group) & (df_raw_data['Phasing'] == phasing), df_raw_data.columns[11]
        ].sum()

        cart_values.append(total_cart)
        order_values.append(total_order)

    df_with_total['加购数'] = cart_values
    df_with_total['下单次数'] = order_values

    # --- Post Total row values ---
    post_total_cart = df_with_total.loc[df_with_total['Phasing'] == 'Post', '加购数'].sum()
    post_total_order = df_with_total.loc[df_with_total['Phasing'] == 'Post', '下单次数'].sum()

    df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', '加购数'] = post_total_cart
    df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', '下单次数'] = post_total_order

    # --- Total row values ---
    during_row = df_with_total.loc[df_with_total['TA GROUP'] == 'During']

    total_cart = post_total_cart + during_row['加购数'].values[0]
    total_order = post_total_order + during_row['下单次数'].values[0]

    df_with_total.loc[df_with_total['TA GROUP'] == 'Total', '加购数'] = total_cart
    df_with_total.loc[df_with_total['TA GROUP'] == 'Total', '下单次数'] = total_order

    # --- Done! Keep as numeric columns, no string formatting ---
    sales_values = []

    for i, row in df_with_total.iterrows():
        ta_group = row['TA GROUP']
        phasing = row['Phasing']

        if ta_group in ["Post Total", "Total"]:
            sales_values.append(0)
            continue

        if ta_group == "During" and phasing == "During":
            total_sales = df_raw_data.loc[df_raw_data['Phasing'] == "During", df_raw_data.columns[12]].sum()  # Column M
            sales_values.append(total_sales)
            continue

        total_sales = df_raw_data.loc[
            (df_raw_data['TA'] == ta_group) & (df_raw_data['Phasing'] == phasing), df_raw_data.columns[12]
        ].sum()

        sales_values.append(total_sales)

    df_with_total['下单sales¥'] = sales_values

    # --- Post Total row value ---
    post_total_sales = df_with_total.loc[df_with_total['Phasing'] == 'Post', '下单sales¥'].sum()
    df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', '下单sales¥'] = post_total_sales

    # --- Total row value ---
    during_row = df_with_total.loc[df_with_total['TA GROUP'] == 'During']

    total_sales = post_total_sales + during_row['下单sales¥'].values[0]
    df_with_total.loc[df_with_total['TA GROUP'] == 'Total', '下单sales¥'] = total_sales

    # --- We now keep as raw float, no string formatting ---
    # If you want to see final table
    df_with_total
    #########################################

    sales_usd_values = []

    for i, row in df_with_total.iterrows():
        ta_group = row['TA GROUP']
        phasing = row['Phasing']

        if ta_group in ["Post Total", "Total"]:
            sales_usd_values.append(0)
            continue

        # Use numeric value directly
        sales_yen_value = row['下单sales¥']  # already numeric, no parsing

        # Convert to USD
        sales_usd = sales_yen_value / currency if sales_yen_value > 0 else 0
        sales_usd_values.append(sales_usd)

    # Assign new numeric column
    df_with_total['下单sales$_num'] = sales_usd_values

    # --- Post Total row value ---
    post_total_sales_usd = df_with_total.loc[df_with_total['Phasing'] == 'Post', '下单sales$_num'].sum()
    df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', '下单sales$_num'] = post_total_sales_usd

    # --- Total row value ---
    during_row = df_with_total.loc[df_with_total['TA GROUP'] == 'During']
    total_sales_usd = post_total_sales_usd + during_row['下单sales$_num'].values[0]
    df_with_total.loc[df_with_total['TA GROUP'] == 'Total', '下单sales$_num'] = total_sales_usd

    # --- You can keep numeric column permanently ---
    # If you'd like to keep both numeric and formatted, do NOT drop numeric column

    # --- Format final display column (optional) ---
    df_with_total['下单sales$'] = df_with_total['下单sales$_num'].apply(lambda x: f"${x:,.2f}" if x > 0 else "")

    # --- Drop helper column only if really needed ---
    # df_with_total = df_with_total.drop(columns=['下单sales$_num'], errors='ignore')

    # --- Show final table ---
    # First, make sure the numeric column is correct (we already have 下单sales$_num)
    # Overwrite 下单sales$ with numeric value directly
    df_with_total['下单sales$'] = df_with_total['下单sales$_num']

    # Drop the helper numeric column
    df_with_total = df_with_total.drop(columns=['下单sales$_num'], errors='ignore')

    # --- Show final table ---
    cols = df_with_total.columns.tolist()

    # Remove duplicates just in case
    cols = list(dict.fromkeys(cols))

    if '下单sales$' in cols and '下单sales¥' in cols:
        # Remove 下单sales$ first
        cols.remove('下单sales$')
        # Find index of 下单sales¥
        sales_yen_index = cols.index('下单sales¥')
        # Insert 下单sales$ before 下单sales¥
        cols = cols[:sales_yen_index] + ['下单sales$'] + cols[sales_yen_index:]

    # Reassign reordered columns
    df_with_total = df_with_total[cols]

    # Show final table

    roi_values = []

    for i, row in df_with_total.iterrows():
        # Get numeric 下单sales¥ directly
        sales_yen_value = row['下单sales¥']
        spend_yen_value = row['Spend￥']

        # Calculate ROI
        if spend_yen_value > 0:
            roi = sales_yen_value / spend_yen_value
        else:
            roi = 0

        roi_values.append(roi)

    # Add numeric ROI column
    df_with_total['下单ROI'] = roi_values
        
##################################################################################
    sales_pay_values = []

    for i, row in df_with_total.iterrows():
        ta_group = row['TA GROUP']
        phasing = row['Phasing']
        if ta_group in ["Post Total", "Total"]:
            sales_pay_values.append(0)
            continue
        if ta_group == "During" and phasing == "During":
            val = df_raw_data.loc[df_raw_data['Phasing'] == "During", df_raw_data.columns[10]].sum()  # Column K
            sales_pay_values.append(val)
            continue
        val = df_raw_data.loc[
            (df_raw_data['TA'] == ta_group) & (df_raw_data['Phasing'] == phasing), df_raw_data.columns[10]
        ].sum()
        sales_pay_values.append(val)

    # --- Add numeric column ---
    df_with_total['付费sales¥'] = sales_pay_values

    # --- Post Total row value ---
    df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', '付费sales¥'] = df_with_total.loc[df_with_total['Phasing'] == 'Post', '付费sales¥'].sum()

    # --- Total row value ---
    df_with_total.loc[df_with_total['TA GROUP'] == 'Total', '付费sales¥'] = (
        df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', '付费sales¥'].values[0]
        + df_with_total.loc[df_with_total['TA GROUP'] == 'During', '付费sales¥'].values[0]
    )

# --- Calculate $ version numerically ---


    df_with_total['付费sales$'] = df_with_total['付费sales¥'] / currency

    roi_pay_values = []

    for i, row in df_with_total.iterrows():
        sales_yen = row['付费sales¥'] if row['付费sales¥'] > 0 else 0
        spend_yen = row['Spend￥'] if row['Spend￥'] > 0 else 0
        roi = sales_yen / spend_yen if spend_yen > 0 else 0
        roi_pay_values.append(roi)

    df_with_total['付费ROI'] = roi_pay_values



    ############################3
    # --- Add numeric column New Customer ---
    new_customer_values = []

    for i, row in df_with_total.iterrows():
        ta_group = row['TA GROUP']
        phasing = row['Phasing']
        if ta_group in ["Post Total", "Total"]:
            new_customer_values.append(0)
            continue
        if ta_group == "During" and phasing == "During":
            val = df_raw_data.loc[df_raw_data['Phasing'] == "During", df_raw_data.columns[19]].sum()
            new_customer_values.append(val)
            continue
        val = df_raw_data.loc[
            (df_raw_data['TA'] == ta_group) & (df_raw_data['Phasing'] == phasing),
            df_raw_data.columns[19]
        ].sum()
        new_customer_values.append(val)

    df_with_total['New Customer'] = new_customer_values

    # --- Post Total row value ---
    df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'New Customer'] = df_with_total.loc[df_with_total['Phasing'] == 'Post', 'New Customer'].sum()

    # --- Total row value ---
    df_with_total.loc[df_with_total['TA GROUP'] == 'Total', 'New Customer'] = (
        df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'New Customer'].values[0]
        + df_with_total.loc[df_with_total['TA GROUP'] == 'During', 'New Customer'].values[0]
)

    


    brand_pay_values = []

    for i, row in df_with_total.iterrows():
        ta_group = row['TA GROUP']
        phasing = row['Phasing']
        if ta_group in ["Post Total", "Total"]:
            brand_pay_values.append(0)
            continue
        if ta_group == "During" and phasing == "During":
            val = df_raw_data.loc[df_raw_data['Phasing'] == "During", df_raw_data.columns[27]].sum()
            brand_pay_values.append(val)
            continue
        val = df_raw_data.loc[
            (df_raw_data['TA'] == ta_group) & (df_raw_data['Phasing'] == phasing),
            df_raw_data.columns[27]
        ].sum()
        brand_pay_values.append(val)

    # Assign directly as final numeric column
    df_with_total['品牌付费sales$'] = brand_pay_values

    # --- Post Total row value ---
    df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', '品牌付费sales$'] = df_with_total.loc[df_with_total['Phasing'] == 'Post', '品牌付费sales$'].sum()

    # --- Total row value ---
    df_with_total.loc[df_with_total['TA GROUP'] == 'Total', '品牌付费sales$'] = (
        df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', '品牌付费sales$'].values[0]
        + df_with_total.loc[df_with_total['TA GROUP'] == 'During', '品牌付费sales$'].values[0]
    )
    
    #############################################3
    brand_ratio_values = []

    for i, row in df_with_total.iterrows():
        ta_group = row['TA GROUP']
        
        # Regular rows
        if ta_group not in ["Post Total", "Total"]:
            brand_sales = row['品牌付费sales$']
            pay_sales = row['付费sales$']
            ratio = brand_sales / pay_sales if pay_sales > 0 else 0
            brand_ratio_values.append(ratio)
            
        # Placeholder for now (we'll fix below)
        else:
            brand_ratio_values.append(0)

    # Assign placeholder values first
    df_with_total['品牌sales%'] = brand_ratio_values

    # Fix "Post Total" row
    if "Post Total" in df_with_total['TA GROUP'].values:
        post_total_brand = df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', '品牌付费sales$'].values[0]
        post_total_pay = df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', '付费sales$'].values[0]
        post_ratio = post_total_brand / post_total_pay if post_total_pay > 0 else 0
        df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', '品牌sales%'] = post_ratio

    # Fix "Total" row
    if "Total" in df_with_total['TA GROUP'].values:
        total_brand = df_with_total.loc[df_with_total['TA GROUP'] == 'Total', '品牌付费sales$'].values[0]
        total_pay = df_with_total.loc[df_with_total['TA GROUP'] == 'Total', '付费sales$'].values[0]
        total_ratio = total_brand / total_pay if total_pay > 0 else 0
        df_with_total.loc[df_with_total['TA GROUP'] == 'Total', '品牌sales%'] = total_ratio

    # Move to left of 品牌付费sales$
    cols = df_with_total.columns.tolist()
    if '品牌sales%' in cols and '品牌付费sales$' in cols:
        cols.remove('品牌sales%')
        idx = cols.index('品牌付费sales$')
        cols = cols[:idx] + ['品牌sales%'] + cols[idx:]
    df_with_total = df_with_total[cols]
    
######################################################  
    brand_roi_values = []

    for i, row in df_with_total.iterrows():
        brand_sales = row['品牌付费sales$']
        spend_usd = row['Spend$']
        
        try:
            roi = brand_sales / spend_usd if spend_usd > 0 else 0
        except:
            roi = 0

        brand_roi_values.append(roi)

    # Assign directly as numeric column
    df_with_total['品牌ROI'] = brand_roi_values

    
#######################################################    
    def parse_currency(x):
        try:
            return float(x.replace("$", "").replace(",", ""))
        except:
            return 0

    df_with_total['Media Budget_num'] = df_with_total['Media Budget'].apply(parse_currency)

    # --- Select value for Post total ---
    post_total_budget = df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'Media Budget_num'].values[0]

    # --- Select value for During ---
    during_budget = df_with_total.loc[df_with_total['TA GROUP'] == 'During', 'Media Budget_num'].values[0]

    # --- Select value for Total ---
    total_budget = df_with_total.loc[df_with_total['TA GROUP'] == 'Total', 'Media Budget_num'].values[0]

    #######################################################  
    # --- Currency rate ---
    new_df = pd.DataFrame()
    new_df['TA GROUP'] = ["TA1", "TA2", "TA3", "TA4", "TA5", "Super TA", "Post Total", "During", "Total"]

    # --- Use Post Total Top-up Internet and Media Budget ---
    post_topup_str = df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'Top-up Internet'].values[0]
    post_topup = float(post_topup_str.replace("¥", "").replace(",", ""))

    post_media_str = df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'Media Budget'].values[0]
    post_media = float(post_media_str.replace("$", "").replace(",", ""))

    # --- Use During Top-up Internet and Media Budget ---
    during_topup_str = df_with_total.loc[df_with_total['TA GROUP'] == 'During', 'Top-up Internet'].values[0]
    during_topup = float(during_topup_str.replace("¥", "").replace(",", ""))

    during_media_str = df_with_total.loc[df_with_total['TA GROUP'] == 'During', 'Media Budget'].values[0]
    during_media = float(during_media_str.replace("$", "").replace(",", ""))

    # --- Currency rate already defined ---
    # currency

    # --- Get Spend$ values for each TA ---
    spend_usd_values = []
    for ta in ["TA1", "TA2", "TA3", "TA4", "TA5", "Super TA", "Post Total", "During", "Total"]:
        try:
            spend_val = df_with_total.loc[df_with_total['TA GROUP'] == ta, 'Spend$'].values[0]
        except:
            spend_val = 0
        spend_usd_values.append(spend_val)

    # --- Calculate 对客Spend$ row by row (numeric) ---
    to_cust_spend_values = []

    for i, ta in enumerate(new_df['TA GROUP']):
        val = 0
        if ta in ["TA1", "TA2", "TA3", "TA4", "TA5", "Super TA"]:
            val = spend_usd_values[i] / post_topup * post_media * currency if post_topup > 0 else 0
        elif ta == "Post Total":
            val = sum(to_cust_spend_values[:6])
        elif ta == "During":
            val = spend_usd_values[i] / during_topup * during_media * currency if during_topup > 0 else 0
        elif ta == "Total":
            val = to_cust_spend_values[-2] + to_cust_spend_values[-1]

        to_cust_spend_values.append(val)

    # --- Add numeric column ---
    new_df['对客Spend$'] = to_cust_spend_values
    #######################################################  
    # --- Use numeric Spend$ directly ---
    df_with_total['Spend$_num'] = df_with_total['Spend$']
    new_df['对客Spend$_num'] = new_df['对客Spend$']

    # --- Get required totals (no rounding needed here if you want exact) ---
    post_total_spend = df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'Spend$_num'].values[0]
    total_spend = df_with_total.loc[df_with_total['TA GROUP'] == 'Total', 'Spend$_num'].values[0]
    total_cust_spend = new_df.loc[new_df['TA GROUP'] == 'Total', '对客Spend$_num'].values[0]
    post_total_redis = new_df.loc[new_df['TA GROUP'] == 'Post Total', '对客Spend$_num'].values[0]

    # --- Prepare list to store values ---
    redis_values = []

    for ta in new_df['TA GROUP']:
        if ta in ['TA1', 'TA2', 'TA3', 'TA4', 'TA5', 'Super TA']:
            orig_spend = df_with_total.loc[df_with_total['TA GROUP'] == ta, 'Spend$_num'].values[0]
            val = orig_spend / post_total_spend * post_total_redis if post_total_spend > 0 else 0
            redis_values.append(val)

        elif ta == 'Post Total':
            val = post_total_spend / total_spend * total_cust_spend if total_spend > 0 else 0
            redis_values.append(val)

        elif ta == 'During':
            during_spend = df_with_total.loc[df_with_total['TA GROUP'] == 'During', 'Spend$_num'].values[0]
            val = during_spend / total_spend * total_cust_spend if total_spend > 0 else 0
            redis_values.append(val)

        elif ta == 'Total':
            val = redis_values[-2] + redis_values[-1]
            redis_values.append(val)

        else:
            redis_values.append(0)

    # --- Add numeric column directly ---
    new_df['综合比例再分配'] = redis_values

    # --- Drop helper columns ---
    df_with_total = df_with_total.drop(columns=['Spend$_num'], errors='ignore')
    new_df = new_df.drop(columns=['对客Spend$_num'], errors='ignore')

        
    
    
    #######################################################
    check_values = []

    for i, ta in enumerate(new_df['TA GROUP']):
        val = 0
        spend_usd = spend_usd_values[i]

        if ta in ["TA1", "TA2", "TA3", "TA4", "TA5", "Super TA"]:
            if post_topup > 0:
                val = spend_usd * (post_media / (post_topup / currency))
        elif ta == "Post Total":
            val = sum(check_values[:6])
        elif ta == "During":
            if during_topup > 0:
                val = spend_usd * (during_media / (during_topup / currency))
        elif ta == "Total":
            val = check_values[-2] + check_values[-1]

        check_values.append(val)

    # --- Add column to new_df
    new_df['check 算法2'] = check_values  
    #######################################################  
    cpm_values = []
    cpc_values = []

    for i, ta in enumerate(new_df['TA GROUP']):
        val_cpm = 0
        val_cpc = 0

        try:
            spend_usd = df_with_total.loc[df_with_total['TA GROUP'] == ta, 'Spend$'].values[0]
        except:
            spend_usd = 0

        try:
            impr_val = df_with_total.loc[df_with_total['TA GROUP'] == ta, 'impression'].values[0]
        except:
            impr_val = 0

        try:
            click_val = df_with_total.loc[df_with_total['TA GROUP'] == ta, 'click'].values[0]
        except:
            click_val = 0

        # CPM calculation
        if impr_val > 0:
            val_cpm = spend_usd / impr_val * 1000

        # CPC calculation
        if click_val > 0:
            val_cpc = spend_usd / click_val

        cpm_values.append(val_cpm)
        cpc_values.append(val_cpc)

    # Add columns to new_df as pure numeric
    new_df['CPM'] = cpm_values
    new_df['CPC'] = cpc_values

    # --- Calculate summary rows ---
    # Post total (index 6)
    post_spend = df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'Spend$'].values[0]
    post_impr = df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'impression'].values[0]
    post_click = df_with_total.loc[df_with_total['TA GROUP'] == 'Post Total', 'click'].values[0]

    post_cpm = post_spend / post_impr * 1000 if post_impr > 0 else 0
    post_cpc = post_spend / post_click if post_click > 0 else 0

    # During (index 7)
    during_spend = df_with_total.loc[df_with_total['TA GROUP'] == 'During', 'Spend$'].values[0]
    during_impr = df_with_total.loc[df_with_total['TA GROUP'] == 'During', 'impression'].values[0]
    during_click = df_with_total.loc[df_with_total['TA GROUP'] == 'During', 'click'].values[0]

    during_cpm = during_spend / during_impr * 1000 if during_impr > 0 else 0
    during_cpc = during_spend / during_click if during_click > 0 else 0

    # Total (index 8)
    total_spend = post_spend + during_spend
    total_impr = post_impr + during_impr
    total_click = post_click + during_click

    total_cpm = total_spend / total_impr * 1000 if total_impr > 0 else 0
    total_cpc = total_spend / total_click if total_click > 0 else 0

    # Update numeric values directly
    new_df.at[6, 'CPM'] = post_cpm
    new_df.at[6, 'CPC'] = post_cpc

    new_df.at[7, 'CPM'] = during_cpm
    new_df.at[7, 'CPC'] = during_cpc

    new_df.at[8, 'CPM'] = total_cpm
    new_df.at[8, 'CPC'] = total_cpc
    
    
    
    #############################33
    
    new_df_to_concat = new_df.drop(columns=['TA GROUP'])

    # Reset index just to be safe (optional)
    new_df_to_concat = new_df_to_concat.reset_index(drop=True)

    # Also reset index of df_with_total to align
    df_with_total = df_with_total.reset_index(drop=True)

    # Concatenate side by side (axis=1)
    df_final = pd.concat([df_with_total, new_df_to_concat], axis=1)

    # Show final result
    df_final = df_final.drop(columns=['Media Budget_num'], errors='ignore')

    #########################
    
    
    
    
    
    
    # ✅ Export full raw df_final first
    #df_final.to_excel("Internal_data_tracker_full.xlsx", index=False)
    #print("✅ Exported full raw data as 'Internal_data_tracker_full.xlsx'!")

    # ✅ Make a deep copy for formatting
    df_internal = df_final.copy(deep=True)

    # --- Move '付费sales$' to left of '付费sales¥'
    cols = df_internal.columns.tolist()
    if '付费sales$' in cols and '付费sales¥' in cols:
        cols.remove('付费sales$')
        idx = cols.index('付费sales¥')
        cols = cols[:idx] + ['付费sales$'] + cols[idx:]
    df_internal = df_internal[cols]

    # ✅ Format currency columns
    currency_columns_dollar = ['下单sales$', '付费sales$', '品牌付费sales$']
    currency_columns_yen = ['下单sales¥', '付费sales¥']

    for col in currency_columns_dollar:
        if col in df_internal.columns:
            df_internal[col] = df_internal[col].apply(lambda x: f"${x:,.2f}" if isinstance(x, (int, float)) and x > 0 else "")

    for col in currency_columns_yen:
        if col in df_internal.columns:
            df_internal[col] = df_internal[col].apply(lambda x: f"¥{x:,.2f}" if isinstance(x, (int, float)) and x > 0 else "")

    # ✅ Format percentage columns
    percent_columns = ['CTR', '品牌sales%']
    for col in percent_columns:
        if col in df_internal.columns:
            df_internal[col] = df_internal[col].apply(lambda x: f"{x:.2%}" if isinstance(x, (int, float)) and x > 0 else "")

    # ✅ Format ROI columns
    for col in ['下单ROI', '付费ROI']:
        if col in df_internal.columns:
            df_internal[col] = df_internal[col].apply(lambda x: f"{x:.1f}" if isinstance(x, (int, float)) and x > 0 else "")

    if '品牌ROI' in df_internal.columns:
        df_internal['品牌ROI'] = df_internal['品牌ROI'].apply(lambda x: f"{x:.1f}" if isinstance(x, (int, float)) and x > 0 else "")

    # ✅ Format CPM and CPC columns
    cpm_cpc_columns = ['CPM', 'CPC']
    for col in cpm_cpc_columns:
        if col in df_internal.columns:
            df_internal[col] = df_internal[col].apply(lambda x: f"${x:,.2f}" if isinstance(x, (int, float)) and x > 0 else "")

    # ✅ Format integer columns
    int_columns = ['impression', 'click', 'PV', 'UV', '加购数', '下单次数', 'New Customer']
    for col in int_columns:
        if col in df_internal.columns:
            df_internal[col] = df_internal[col].apply(lambda x: f"{int(x):,}" if isinstance(x, (int, float)) and x > 0 else "")

    # --- Dollar columns rounded to whole number
    whole_number_dollar_columns = ['Spend$', '对客Spend$', '综合比例再分配', 'check', '算法2']
    for col in whole_number_dollar_columns:
        if col in df_internal.columns:
            df_internal[col] = df_internal[col].apply(lambda x: f"${round(x):,}" if isinstance(x, (int, float)) and x > 0 else "")

    # --- Yen columns keep 2 decimals
    yen_columns = ['Spend￥']
    for col in yen_columns:
        if col in df_internal.columns:
            df_internal[col] = df_internal[col].apply(lambda x: f"¥{x:,.2f}" if isinstance(x, (int, float)) and x > 0 else "")

    # --- Rename column for final presentation
    df_internal = df_internal.rename(columns={'impression': 'Impression'})
    
    ###################################


    df_final_copy_12 = pd.DataFrame()
    df_final_copy_12 = df_final.copy(deep=True)

    df_final_copy_12 = df_final_copy_12.drop(
        columns=[
            "Spend￥", "Top-up Internet", "PV", "UV", "加购数", "下单次数", 
            "下单sales$", "下单sales¥", "下单ROI", "付费sales¥", "付费sales$", 
            "付费ROI", "New Customer", "Spend$", "对客Spend$", "check 算法2",
            "CPM", "CPC", "品牌付费sales$", "CTR"
        ],
        errors='ignore'
    )

    # --- Rename ---
    df_final_copy_12 = df_final_copy_12.rename(columns={'综合比例再分配': 'Spend'})

    # --- Move Spend left of impression ---
    cols = df_final_copy_12.columns.tolist()
    if 'Spend' in cols and 'impression' in cols:
        cols.remove('Spend')
        idx = cols.index('impression')
        cols = cols[:idx] + ['Spend'] + cols[idx:]

    df_final_copy_12 = df_final_copy_12[cols]
    
    ###################################
    def parse_currency(x):
        try:
            return float(x.replace("$", "").replace(",", ""))
        except:
            return 0

    # --- Convert columns ---
    df_final_copy_12['Media Budget_num'] = df_final_copy_12['Media Budget'].apply(parse_currency)
    df_final_copy_12['Spend_num'] = df_final_copy_12['Spend']

    # --- Get correct group budget values ---
    post_total_media_budget = df_final_copy_12.loc[df_final_copy_12['TA GROUP'] == 'Post Total', 'Media Budget_num'].values[0]
    during_media_budget = df_final_copy_12.loc[df_final_copy_12['TA GROUP'] == 'During', 'Media Budget_num'].values[0]
    total_media_budget = df_final_copy_12.loc[df_final_copy_12['TA GROUP'] == 'Total', 'Media Budget_num'].values[0]

    # --- Calculate progress ---
    progress_values = []

    for i, row in df_final_copy_12.iterrows():
        spend = row['Spend_num']
        ta_group = row['TA GROUP']
        
        progress = ""

        if ta_group == "Post Total" and post_total_media_budget > 0:
            progress = spend / post_total_media_budget
        elif ta_group == "During" and during_media_budget > 0:
            progress = spend / during_media_budget
        elif ta_group == "Total" and total_media_budget > 0:
            progress = spend / total_media_budget

        if isinstance(progress, float) and progress > 0:
            progress = f"{progress:.2%}"
        else:
            progress = ""

        progress_values.append(progress)

    # --- Drop any old Spending Progress columns (case insensitive, robust) ---
    df_final_copy_12 = df_final_copy_12.loc[:, ~df_final_copy_12.columns.str.contains('Spending Progress', case=False, regex=False)]

    # --- Add new column ---
    df_final_copy_12['Spending Progress'] = progress_values

    # --- Drop helper columns ---
    df_final_copy_12 = df_final_copy_12.drop(columns=['Media Budget_num', 'Spend_num'], errors='ignore')




    ######################################
    # --- Calculate numeric CTR ---
    df_final_copy_12['CTR'] = df_final_copy_12['click'] / df_final_copy_12['impression']

    # --- Replace NaN or inf with 0 (just in case) ---
    df_final_copy_12['CTR'] = df_final_copy_12['CTR'].fillna(0).replace([float('inf'), -float('inf')], 0)

    # --- Move CTR column next to click ---
    cols = df_final_copy_12.columns.tolist()
    if 'click' in cols and 'CTR' in cols:
        cols.remove('CTR')
        idx = cols.index('click')
        cols = cols[:idx + 1] + ['CTR'] + cols[idx + 1:]

    df_final_copy_12 = df_final_copy_12[cols]

    ##########################################
    cols = df_final_copy_12.columns.tolist()

    if 'Media Budget' in cols and 'Spending Progress' in cols:
        # Remove 'Spending Progress' first
        cols.remove('Spending Progress')
        # Find index of 'Media Budget'
        idx = cols.index('Media Budget')
        # Insert after
        cols = cols[:idx + 1] + ['Spending Progress'] + cols[idx + 1:]

    df_final_copy_12 = df_final_copy_12[cols]

    # ✅ Done
    df_final_copy_12

    df_final_copy_12['CPC'] = df_final_copy_12['Spend'] / df_final_copy_12['click']

    # --- Replace NaN or inf with 0 (just in case) ---
    df_final_copy_12['CPC'] = df_final_copy_12['CPC'].fillna(0).replace([float('inf'), -float('inf')], 0)

    # --- Move CPC to right of CPM if needed ---
    cols = df_final_copy_12.columns.tolist()
    if 'CPM' in cols and 'CPC' in cols:
        cols.remove('CPC')
        idx = cols.index('CPM')
        cols = cols[:idx + 1] + ['CPC'] + cols[idx + 1:]

    df_final_copy_12 = df_final_copy_12[cols]

#####################################################

    # --- Use numeric impression and click directly ---
    df_final_copy_12['impression_num'] = df_final_copy_12['impression']
    df_final_copy_12['click_num'] = df_final_copy_12['click']

    # --- Calculate CPM ---
    df_final_copy_12['CPM'] = df_final_copy_12.apply(
        lambda row: row['Spend'] / row['impression_num'] * 1000 if row['impression_num'] > 0 else 0,
        axis=1
    )

    # --- Calculate CPC ---
    df_final_copy_12['CPC'] = df_final_copy_12.apply(
        lambda row: row['Spend'] / row['click_num'] if row['click_num'] > 0 else 0,
        axis=1
    )

    # --- Drop helper columns ---
    df_final_copy_12 = df_final_copy_12.drop(columns=['impression_num', 'click_num'], errors='ignore')

    # --- Move CPM right of CTR, CPC right of CPM ---
    cols = df_final_copy_12.columns.tolist()

    if 'CTR' in cols and 'CPM' in cols and 'CPC' in cols:
        cols.remove('CPM')
        cols.remove('CPC')
        idx = cols.index('CTR')
        cols = cols[:idx + 1] + ['CPM', 'CPC'] + cols[idx + 1:]

    df_final_copy_12 = df_final_copy_12[cols]

    # --- Rename columns ---
    df_final_copy_12 = df_final_copy_12.rename(columns={'品牌sales%': 'Brand Sales%', "品牌ROI": "Brand Transcation ROAS"})

######################################################
    brand_trans_sales_values = []

    for i, row in df_final_copy_12.iterrows():
        ta_group = row['TA GROUP']
        spend = row['Spend']
        roas = row['Brand Transcation ROAS']
        
        amount = 0

        if ta_group in ['TA1', 'TA2', 'TA3', 'TA4', 'TA5', 'Super TA']:
            amount = roas * spend
            brand_trans_sales_values.append(amount)
        elif ta_group == 'Post Total':
            amount = sum(brand_trans_sales_values)
            brand_trans_sales_values.append(amount)
        elif ta_group == 'During':
            amount = roas * spend
            brand_trans_sales_values.append(amount)
        elif ta_group == 'Total':
            amount = brand_trans_sales_values[-2] + brand_trans_sales_values[-1]
            brand_trans_sales_values.append(amount)
        else:
            brand_trans_sales_values.append(0)

    # --- Add numeric column directly ---
    df_final_copy_12['Brand Transaction Sales Amount'] = brand_trans_sales_values


######################################################
    backend_amount_values = []

    for i, row in df_final_copy_12.iterrows():
        trans_amount = row['Brand Transaction Sales Amount']
        brand_pct = row['Brand Sales%']
        
        if brand_pct > 0:
            backend_amount = trans_amount / brand_pct
        else:
            backend_amount = 0

        backend_amount_values.append(backend_amount)

    # --- Add new numeric column directly ---
    df_final_copy_12['Backend Transaction Amount'] = backend_amount_values



#####################################################
    df_final_copy_12['Brand Transaction Sales Amount'] = df_final_copy_12['Brand Transcation ROAS'] * df_final_copy_12['Spend']

    # --- Calculate Backend Transaction Amount ---
    df_final_copy_12['Backend Transaction Amount'] = df_final_copy_12['Brand Transaction Sales Amount'] / df_final_copy_12['Brand Sales%']

    # --- Brand Average Discount Rate ---
    discount_rate_list = []
    for i, row in df_final_copy_12.iterrows():
        if row['Phasing'] == 'Post':
            discount_rate_list.append(1 - post_discount_rate)
        elif row['Phasing'] == 'During':
            discount_rate_list.append(1 - during_discount_rate)
        elif row['TA GROUP'] == 'Total':
            discount_rate_list.append(None)
        else:
            discount_rate_list.append(1 - post_discount_rate)

    df_final_copy_12['Brand Average Discount Rate'] = discount_rate_list

    # --- Brand SRP ROAS ---
    srp_roas_list = []
    for i, row in df_final_copy_12.iterrows():
        if row['TA GROUP'] in ['TA1', 'TA2', 'TA3', 'TA4', 'TA5', 'Super TA']:
            val = row['Brand Transcation ROAS'] / post_discount_rate
            srp_roas_list.append(val)
        elif row['TA GROUP'] == 'During':
            val = row['Brand Transcation ROAS'] / during_discount_rate
            srp_roas_list.append(val)
        else:
            srp_roas_list.append(0)

    df_final_copy_12['Brand SRP ROAS'] = srp_roas_list

    # --- Post Total weighted SRP ROAS ---
    post_rows = df_final_copy_12[df_final_copy_12['Phasing'] == 'Post']
    post_spend_sum = post_rows['Spend'].sum()
    post_weighted_srp = (post_rows['Spend'] * post_rows['Brand SRP ROAS']).sum() / post_spend_sum

    df_final_copy_12.loc[df_final_copy_12['TA GROUP'] == 'Post Total', 'Brand SRP ROAS'] = post_weighted_srp

    # --- Total SRP ROAS ---
    during_row = df_final_copy_12[df_final_copy_12['TA GROUP'] == 'During']
    during_spend = during_row['Spend'].values[0]
    during_srp = during_row['Brand SRP ROAS'].values[0]
    total_spend = df_final_copy_12.loc[df_final_copy_12['TA GROUP'] == 'Total', 'Spend'].values[0]

    total_srp_roas = (post_weighted_srp * post_spend_sum + during_srp * during_spend) / total_spend
    df_final_copy_12.loc[df_final_copy_12['TA GROUP'] == 'Total', 'Brand SRP ROAS'] = total_srp_roas

    # --- Total Brand Average Discount Rate ---
    total_bt_roas = df_final_copy_12.loc[df_final_copy_12['TA GROUP'] == 'Total', 'Brand Transcation ROAS'].values[0]
    total_avg_discount = 1 - (total_bt_roas / total_srp_roas)
    df_final_copy_12.loc[df_final_copy_12['TA GROUP'] == 'Total', 'Brand Average Discount Rate'] = total_avg_discount

    # ✅ Columns now all remain numeric until export or display
    

########################
    # Get columns, keeping only first occurrence
    _, idx = np.unique(df_final_copy_12.columns, return_index=True)

    # Reindex columns to keep only first
    df_final_copy_12 = df_final_copy_12.iloc[:, np.sort(idx)]
    # --- Rename column ---
    # --- Rename column first ---
    df_final_copy_12 = df_final_copy_12.rename(columns={'impression': 'Impression','Media Budget':'Planned Media Budget','TA 定义':'Name'})

    # --- Desired front columns from your request ---
    front_columns = [
        'Campaign Name', 
        'Brand', 
        'Date', 
        'Phasing', 
        'TA GROUP', 
        'Name', 
        'Planned Media Budget', 
        'Spending Progress', 
        'Spend',
        'Impression',
        'click',
        'CTR',
        'CPM',
        'CPC',
        'Backend Transaction Amount',
        'Brand Sales%',
        'Brand Average Discount Rate',
        "Brand Transaction Sales Amount",
        "Brand Transcation ROAS"


    
    ]

    # --- Get all existing columns in the current dataframe ---
    all_columns = df_final_copy_12.columns.tolist()

    # --- Identify columns not in front_columns ---
    remaining_columns = [col for col in all_columns if col not in front_columns]

    # --- Final column order: front columns first, then remaining ones ---
    final_columns = front_columns + remaining_columns

    # --- Reorder dataframe ---
    df_final_copy_12 = df_final_copy_12[final_columns]

    # --- Display final dataframe ---




    
#######################

    if df_raw_data_path_previous_week and os.path.exists(df_raw_data_path_previous_week):
        df_raw_data_previous_week = pd.read_excel(df_raw_data_path_previous_week)
        df_final_copy_12_previous = internal_data_tracker(
            currency,
            df_raw_data_previous_week,
            post_discount_rate,
            during_discount_rate
        )
        # ✅ Export formatted file
        df_internal.to_excel("Current_Week_Internal_data_tracker.xlsx", index=False)
        print("✅ Exported formatted version as 'Current_Week_Internal_data_tracker.xlsx'!")

        df_final_copy_12.to_excel("Current_week_culmulative_table.2.0.xlsx", index=False)
        print("✅ File 'Current_week_culmulative_table.2.0.xlsx' has been created successfully!")
        return df_final_copy_12, df_final_copy_12_previous
    else:
        df_internal.to_excel("Current_Week_Internal_data_tracker.xlsx", index=False)
        print("✅ Exported formatted version as 'Current_Week_Internal_data_tracker.xlsx'!")

        df_final_copy_12.to_excel("Current_week_culmulative_table.2.0.xlsx", index=False)
        print("✅ File 'Current_week_culmulative_table.2.0.xlsx' has been created successfully!")
      
        return df_final_copy_12



In [2]:
result = internal_data_tracker(
    currency,
    df_raw_data_current_week,
    post_discount_rate,
    during_discount_rate,
    df_raw_data_path_previous_week=path_previous
)
if isinstance(result, tuple):
    df_current, df_previous = result
else:
    df_current = result
    df_previous = None

✅ Exported formatted version as 'Current_Week_Internal_data_tracker.xlsx'!
✅ File 'Current_week_culmulative_table.2.0.xlsx' has been created successfully!
✅ Exported formatted version as 'Current_Week_Internal_data_tracker.xlsx'!
✅ File 'Current_week_culmulative_table.2.0.xlsx' has been created successfully!


In [3]:
df_current

,Campaign Name,Brand,Date,Phasing,TA GROUP,Name,Planned Media Budget,Spending Progress,Spend,Impression,click,CTR,CPM,CPC,Backend Transaction Amount,Brand Sales%,Brand Average Discount Rate,Brand Transaction Sales Amount,Brand Transcation ROAS,Brand SRP ROAS
0,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,TA1,TA1 定义,"$666,077",,63871.641971,2302571,38377,0.016667,27.739271,1.664321,6.594846e+05,0.259983,0.230000,1.714547e+05,2.684364,3.486187
1,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,TA2,TA2 定义,,,177.277784,8801,132,0.014998,20.142914,1.343014,1.812684e+03,0.246320,0.230000,4.465009e+02,2.518651,3.270976
2,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,TA3,TA3 定义,,,39791.549251,2924074,70103,0.023974,13.608257,0.567615,5.059943e+05,0.259445,0.230000,1.312775e+05,3.299131,4.284586
3,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,TA4,TA4 定义,,,37900.904370,2019593,43260,0.021420,18.766605,0.876119,4.719476e+05,0.257302,0.230000,1.214330e+05,3.203960,4.160988
4,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,TA5,TA5 定义,,,117307.147036,8406794,269230,0.032025,13.953851,0.435714,1.771341e+06,0.243795,0.230000,4.318433e+05,3.681304,4.780915
5,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,Super TA,Super TA 定义,,,291295.238469,38963682,2203623,0.056556,7.476071,0.132189,5.327871e+06,0.249033,0.230000,1.326817e+06,4.554886,5.915437
6,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,Post Total,,"$666,077",82.63%,550363.922354,54625515,2624725,0.048049,10.075217,0.209684,8.738771e+06,0.249847,0.230000,2.183352e+06,3.967106,2.575995
7,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,During,During,Hainan Traveler,"$311,560",89.68%,279392.268174,32900721,823384,0.025026,8.491980,0.339322,4.079345e+06,0.248610,0.320000,1.014167e+06,3.629904,5.338094
8,,,,,Total,,"$977,637",84.87%,829756.190528,87526236,3448109,0.039395,9.480085,0.240641,1.281812e+07,0.249453,0.261004,3.197519e+06,3.853564,5.214592


In [4]:
df_previous

,Campaign Name,Brand,Date,Phasing,TA GROUP,Name,Planned Media Budget,Spending Progress,Spend,Impression,click,CTR,CPM,CPC,Backend Transaction Amount,Brand Sales%,Brand Average Discount Rate,Brand Transaction Sales Amount,Brand Transcation ROAS,Brand SRP ROAS
0,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,TA1,TA1 定义,"$666,077",,61051.072677,1846456,36225,0.019619,33.063920,1.685330,6.332647e+05,0.262959,0.230000,1.665228e+05,2.727598,3.542335
1,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,TA2,TA2 定义,,,177.277784,8801,132,0.014998,20.142914,1.343014,1.812684e+03,0.246320,0.230000,4.465009e+02,2.518651,3.270976
2,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,TA3,TA3 定义,,,37785.178214,2759573,68995,0.025002,13.692400,0.547651,4.876664e+05,0.262126,0.230000,1.278301e+05,3.383074,4.393603
3,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,TA4,TA4 定义,,,37058.396598,1999144,42589,0.021304,18.537132,0.870140,4.644684e+05,0.258416,0.230000,1.200262e+05,3.238839,4.206284
4,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,TA5,TA5 定义,,,97521.877919,6552300,206470,0.031511,14.883610,0.472330,1.500343e+06,0.253854,0.230000,3.808687e+05,3.905470,5.072038
5,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,Super TA,Super TA 定义,,,260680.006030,30226912,1799726,0.059541,8.624103,0.144844,4.704365e+06,0.257109,0.230000,1.209535e+06,4.639923,6.025874
6,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,Post Total,,"$666,077",74.21%,494291.461844,43393186,2154137,0.049642,11.390993,0.229461,7.792199e+06,0.257347,0.230000,2.005301e+06,4.056920,2.634317
7,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,During,During,Hainan Traveler,"$311,560",77.52%,241509.752613,27004529,698344,0.025860,8.943305,0.345832,3.561058e+06,0.253517,0.320000,9.027872e+05,3.738098,5.497203
8,,,,,Total,,"$977,637",75.26%,735801.214456,70397715,2852481,0.040520,10.452061,0.257951,1.135326e+07,0.256146,0.260371,2.908088e+06,3.952274,5.343593


In [5]:
def weekly_report (df_current,df_previous):
    if df_previous is None:
        return df_current
    else:
        df_weekly = df_current.copy(deep = True)
        df_weekly = df_weekly.drop(columns=['Planned Media Budget', 'Spending Progress'], errors='ignore')
        df_weekly =  df_weekly.rename(columns={'Spend': 'Weekly Spend'})
        df_weekly['Weekly Spend'] = df_current['Spend'] - df_previous['Spend']
        df_weekly['Impression'] = df_current['Impression'] - df_previous['Impression']
        df_weekly['click'] = df_current['click'] - df_previous['click']
        df_weekly['CTR'] = df_weekly.apply(
            lambda row: row['click'] / row['Impression'] if row['Impression'] > 0.00000000001 else 0,
            axis=1
        )
        df_weekly['CPM'] = df_weekly.apply(
            lambda row: row['Weekly Spend'] / row['Impression'] * 1000 if row['Impression'] > 0.00000000001 else 0,
            axis=1
        )

        df_weekly['CPC'] = df_weekly.apply(
            lambda row: row['Weekly Spend'] / row['click'] if row['click'] > 0.00000000001 else 0,
            axis=1
        )

        df_weekly['Backend Transaction Amount'] = df_current['Backend Transaction Amount'] - df_previous['Backend Transaction Amount']

        df_weekly['Brand Transaction Sales Amount'] = df_current['Brand Transaction Sales Amount'] - df_previous['Brand Transaction Sales Amount']


        df_weekly['Brand Sales%'] = df_weekly.apply(
            lambda row: row['Brand Transaction Sales Amount'] / row['Backend Transaction Amount'] if row['Backend Transaction Amount'] > 0.00000000001 else 0,
            axis=1
        )

        df_weekly['Brand Transcation ROAS'] = df_weekly.apply(
            lambda row: row['Brand Transaction Sales Amount'] / row['Weekly Spend'] if row['Weekly Spend'] > 0.00000000001 else 0,
            axis=1
        )


        df_weekly['Brand SRP ROAS'] = df_weekly.apply(
            lambda row: row['Brand Transcation ROAS'] / (1-row['Brand Average Discount Rate']) if (1-row['Brand Average Discount Rate']) > 0.00000000001 else 0,
            axis=1
        )

        df_weekly['Backend Transactional ROAS'] = df_weekly.apply(
            lambda row: row['Backend Transaction Amount'] / row['Weekly Spend'] if row['Weekly Spend'] > 0.00000000001 else 0,
            axis=1
        )

        df_weekly['Backend Transactional ROAS'] = df_weekly.apply(
        lambda row: row['Backend Transaction Amount'] / row['Weekly Spend'] if row['Weekly Spend'] > 0.00000000001 else 0,
        axis=1
    )

    # ✅ Move 'Backend Transaction Amount' to the rightmost column
        cols = df_weekly.columns.tolist()
        if 'Backend Transaction Amount' in cols:
            cols.remove('Backend Transaction Amount')
            cols.append('Backend Transaction Amount')
            df_weekly = df_weekly[cols]

        df_weekly.to_excel("Current_weekly_table.xlsx", index=False)
        print("✅ File 'Current_weekly_table.xlsx' has been created successfully!")


        return df_weekly


In [6]:
weekly_report(df_current,df_previous)

✅ File 'Current_weekly_table.xlsx' has been created successfully!


,Campaign Name,Brand,Date,Phasing,TA GROUP,Name,Weekly Spend,Impression,click,CTR,CPM,CPC,Brand Sales%,Brand Average Discount Rate,Brand Transaction Sales Amount,Brand Transcation ROAS,Brand SRP ROAS,Backend Transactional ROAS,Backend Transaction Amount
0,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,TA1,TA1 定义,2.820569e+03,456115,2152,0.004718,6.183899,1.310673,0.188100,0.230000,4.931959e+03,1.748569,2.270868,9.295955,2.621989e+04
1,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,TA2,TA2 定义,2.842171e-14,0,0,0.000000,0.000000,0.000000,0.000000,0.230000,5.684342e-14,0.000000,0.000000,0.000000,2.273737e-13
2,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,TA3,TA3 定义,2.006371e+03,164501,1108,0.006736,12.196710,1.810804,0.188100,0.230000,3.447471e+03,1.718262,2.231509,9.134825,1.832785e+04
3,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,TA4,TA4 定义,8.425078e+02,20449,671,0.032813,41.200439,1.255600,0.188099,0.230000,1.406833e+03,1.669816,2.168593,8.877311,7.479203e+03
4,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,TA5,TA5 定义,1.978527e+04,1854494,62760,0.033842,10.668823,0.315253,0.188100,0.230000,5.097457e+04,2.576390,3.345961,13.696918,2.709972e+05
5,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,Super TA,Super TA 定义,3.061523e+04,8736770,403897,0.046230,3.504182,0.075800,0.188100,0.230000,1.172815e+05,3.830821,4.975092,20.365879,6.235061e+05
6,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,Post,Post Total,,5.607246e+04,11232329,470588,0.041896,4.992060,0.119154,0.188100,0.230000,1.780507e+05,3.175368,4.123854,16.881231,9.465721e+05
7,LM FY25 Growth Hunting - Q4,LM,4/1–6/30,During,During,Hainan Traveler,3.788252e+04,5896192,125040,0.021207,6.424912,0.302963,0.214900,0.320000,1.113799e+05,2.940140,4.323736,13.681424,5.182868e+05
8,,,,,Total,,9.395498e+04,17128521,595628,0.034774,5.485294,0.157741,0.197583,0.261004,2.894306e+05,3.080524,4.168525,15.591073,1.464859e+06
